In [ ]:
import glob

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, display  # type: ignore

from automatic_climb_detection.resample_dataframe import resample_dataframe

# Loading the data
Read a route from the 15th Tour de France. 

It might be easier for us to work with data that is always taken at the same interval.

Therefore, we resample the original `DataFrame` to a new `DataFrame` that has latitude, longitude, and altitude information every 1000 meters.

In [ ]:
file = "../data/TourDeFrance2022/stage-15-parcours.csv"
df = pd.read_csv(file, index_col=0)
df1km = resample_dataframe(df, interpolation_column="distance", interpolation_step=1000)

# Plotting

Let's create a simple plot to visualize the data. 

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
plt.suptitle(file)
axs[0].plot(df["longitude"], df["latitude"], "-o", label="High resolution data")
axs[0].plot(df1km["longitude"], df1km["latitude"], "-o", label="Low resolution data")
axs[0].set(xlabel="Longitude", ylabel="Latitude")
axs[0].legend()
axs[1].plot(df["distance"], df["altitude"], "-o", label="High resolution data")
axs[1].plot(df1km["distance"], df1km["altitude"], "-o", label="Low resolution data")
axs[1].set(xlabel="Distance [m]", ylabel="Altitude [m]")
axs[1].legend()

# Adding a map
We can plot the route on an interactive map.

In [ ]:
map_obj = folium.Map(
    location=(48.856, 2.3522),  # Paris
    zoom_start=5,
    width="100%",
    height="100%",
)

csv_files = glob.glob("../data/TourDeFrance2022/*.csv")
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df = resample_dataframe(
        df, interpolation_column="distance", interpolation_step=1000
    )
    # Convert the latitude and longitude columns to a 'geometry' column
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    # Set the coordinate reference system (CRS) for the GeoDataFrame
    gdf.crs = "EPSG:4326"
    # Create a list of tuples containing latitude and longitude pairs
    route_coords = list(zip(gdf.latitude, gdf.longitude))
    # Add a polyline with the coordinates of the route
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.7).add_to(map_obj)


# Set the output cell width to match the map width
display(HTML("<style>.output {width: 40% !important;}</style>"))
# Display the map in Jupyter notebook
map_obj